Based on code: http://lethain.com/genetic-algorithms-cool-name-damn-simple/

Function to solve: http://www.pi.zarz.agh.edu.pl/tematy/EA/labor/ea_zad2.html

In [20]:
from random import randint, random
from operator import add
from functools import reduce
from numpy.random import choice
import string
import numpy as np
import math

#### Create member of population and population 

In [82]:
def individual(min, max):
     return [ randint(min,max) for x in range(2)] #range(2) for 2 parameters

In [83]:
def population(count, min, max):
    return[individual(min, max) for x in range(count)]

#### Fitness function 

Searching for local minimum in space [-2.047;2.047]

In [4]:
def fitness(individual):
    fx=(100*pow((pow(individual[0]*0.001,2))-individual[1]*0.001,2)
    +pow((1-individual[0]*0.001),2))
    return fx  #liczba ujemna mniejsza 

#### Population parameters 

In [5]:
def grade(pop): #średni błąd dla populacji
    summed= reduce(add, (fitness(x) for x in pop),0)
    return(summed/(len(pop)*1.0))

#### new population of parents - roulette selection

In [102]:
def roulette_selection(lista):
    fitness_value = [ (fitness(x), x) for x in lista] #fit function and indifidual for each individual
    
    def rank(value):
        return (1/math.sqrt(value))
    
    fitness_sum = sum(rank(fit) for fit,x in fitness_value)
    weights =[rank(fit)/fitness_sum for fit,x in fitness_value]
    parents_length=len(weights)
    parents_indicator = choice(range(parents_length),int(parents_length),p=weights) 
    selected_parents=[fitness_value[i][1] for i in parents_indicator]
    return (selected_parents)

In [90]:
def convert_to_binary(data, bit_length):
    return [(format(x,bit_length),format(y,bit_length)) for x,y in data]

## Breeding & mutation rules 

In [132]:
def evolve(pop, retain=0.2, mutate=0.1):
    
    graded = [ (fitness(x), x) for x in pop] #fit function and indifidual for each individual
    graded = [ x[1] for x in sorted(graded)] #above, sorted by fitness (lowest to highest)
    retain_length = int(len(graded)*retain) #select indifiduals to stay unchaanged
    
    ### retain some parents
    retained_parents=[]
    retained_parents = graded[:retain_length] 
    not_retained_parents = graded[retain_length:]
    
    new_parents=roulette_selection(not_retained_parents)
    
    new_parents_binary=convert_to_binary(new_parents,bit_length="014b")
    
    ###crossover
    n_of_bits=len(new_parents_binary[0][0])
    length_new_parents=len(new_parents)
    ind1 = choice(range(length_new_parents),size=length_new_parents,replace=False) #  point out on new_parents_binary
    split_point=choice(n_of_bits-4,size=int(length_new_parents/2),replace=True)+2  # create split point for each parent

    # if length_new_parents is even, take middle value and append to parents_to retain
    if (int(length_new_parents/2))%1!=0:
        to_append=new_parents_binary[int(length_new_parents/2)+1]
        to_append=[int(to_append[0],2),int(to_append[1],2)]
        retained_parents.append(to_append)
        
    #crossover
    for i in range(int(length_new_parents/2)):
        parent1,parent2=new_parents_binary[i-1],new_parents_binary[length_new_parents-i-1]
        child1=(parent1[0][:split_point[i]]+parent2[0][split_point[i]:],parent1[1][:split_point[i]]+parent2[1][split_point[i]:])
        child2=(parent2[0][:split_point[i]]+parent1[0][split_point[i]:],parent2[1][:split_point[i]]+parent1[1][split_point[i]:])
        
        # mutate with proabbility="mutate"
        if choice([0,1], 1, p=[1-mutate,mutate])==1:            
            index1=randint(3,n_of_bits-1)
            child1=(child1[0][:index1] + str("1") + child1[0][index1 + 1:],child1[1]) # change only 1st. element of 1st. child
        if choice([0,1], 1, p=[1-mutate,mutate])==1:
            index2=randint(3,n_of_bits-1)      
            child2=(child2[0],child2[1][:index2] + str("1") + child2[1][index2 + 1:])# change only 2st. element of 2st. child
        #change to integer
        int_child1=[int(child1[0],2),int(child1[1],2)]
        int_child2=[int(child2[0],2),int(child2[1],2)]        
        retained_parents.append(int_child1)
        retained_parents.append(int_child2)
    return (retained_parents)

## Target


In [138]:
p_count = 50
i_min = -2047
i_max = 2047
p = population(p_count, i_min, i_max)
fitness_history = [grade(p),]
for i in range(50):
    populacja = p
    p = evolve(p)
    fitness_history.append(grade(p))
    
for datum in fitness_history:
    print (datum)
print("population: \n" ,populacja)

392.8112719546079
377.098643353222
154.88238073572404
115.02056838705799
113.49136200378801
86.17700172312401
35.000082101051994
16.484315437792002
27.960055323490007
6.762841158338
3.321069980546001
4.922431311036
36.39791194312802
2.378274663258
5.982320675294
7.9850491137559985
0.5603598901340002
1.429974746576
1.3696635163959998
1.1958837327119998
0.5316938512460001
7.791998347059998
0.8273687425559999
0.29304430753599997
0.29691421140599994
0.8370837293519995
7.559477736065999
0.2916472890879999
0.434652909848
7.703447555191998
0.2933122453419999
1.3696791412120002
0.29253309523599996
0.8312735323520002
0.837193140056
0.29124644102599995
0.2913609759960001
3.364636012544003
0.3012217896540001
8.699233989779998
0.8423482953820002
8.527099749307997
0.3034578228440001
0.3072316474820002
2.556598506418
0.2899185250080002
0.2881334248840001
1.246622812662
0.2899927452420002
0.29753545970200024
2.9575950371380006
population: 
 [[1007, 1014], [889, 791], [877, 763], [1017, 1016], [877, 7